In [1]:
import pandas as pd

In [2]:
ad_clicks = pd.read_csv('ad_clicks.csv')
print (ad_clicks.head(5))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


How many views came from each utm_source?

In [4]:
views = ad_clicks.groupby('utm_source').user_id.count().reset_index()
print(views)

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


Adding a columnn "is_click" when the ad has been clicked on:


In [6]:
ad_clicks['is_click']= ~ad_clicks.ad_click_timestamp.isnull()

Find out the percentage of people that clicked on ads from each utm_source:

In [9]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
print(clicks_by_source)

  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66


In [13]:
clicks_pivot= clicks_by_source.pivot(
columns = 'is_click',
index = 'utm_source',
values = 'user_id')

clicks_pivot['percent_clicked'] = round((clicks_pivot[True] / (clicks_pivot[True]+ clicks_pivot[False]))*100)
print(clicks_pivot)

is_click    False  True  percent_clicked
utm_source                              
email         175    80             31.0
facebook      324   180             36.0
google        441   239             35.0
twitter       149    66             31.0


Analyzing A/B test:

-> checking if approximately the same amount of people were shown both ads?

-> which ad was clicked on by the greater percentage of users?

In [15]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

print (a_clicks.user_id.count(), b_clicks.user_id.count())

827 827


In [48]:
A_B_clicks = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()

A_B_pivot = A_B_clicks.pivot(
columns = 'is_click',
index = 'experimental_group',
values = 'user_id').reset_index()

A_B_pivot['percent_clicked'] = round((A_B_pivot[True] /( A_B_pivot[True]+ A_B_pivot[False]))*100)

print(A_B_pivot)

is_click experimental_group  False  True  percent_clicked
0                         A    517   310             37.0
1                         B    572   255             31.0


In [46]:
a_clicks_pivot = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
columns = 'is_click',
index = 'day',
values = 'user_id').reset_index()

a_clicks_pivot['percent_clicked'] = round((a_clicks_pivot[True] / (a_clicks_pivot[True]+ a_clicks_pivot[False]))*100)

print(a_clicks_pivot)

is_click            day  False  True  percent_clicked
0            1 - Monday     70    43             38.0
1           2 - Tuesday     76    43             36.0
2         3 - Wednesday     86    38             31.0
3          4 - Thursday     69    47             41.0
4            5 - Friday     77    51             40.0
5          6 - Saturday     73    45             38.0
6            7 - Sunday     66    43             39.0


In [47]:
b_clicks_pivot = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
columns = 'is_click',
index = 'day',
values = 'user_id').reset_index()

b_clicks_pivot['percent_clicked'] = round((b_clicks_pivot[True] / (b_clicks_pivot[True]+ b_clicks_pivot[False]))*100)

print(b_clicks_pivot)

is_click            day  False  True  percent_clicked
0            1 - Monday     81    32             28.0
1           2 - Tuesday     74    45             38.0
2         3 - Wednesday     89    35             28.0
3          4 - Thursday     87    29             25.0
4            5 - Friday     90    38             30.0
5          6 - Saturday     76    42             36.0
6            7 - Sunday     75    34             31.0


Ad "A" is recommended as it was more popular among shoppers overall (by 6%), as well as was constantly over 30% of clicks during the week, working especially well closer to the end of the week.